In [1]:
import os, sys
os.getcwd()
#sys.path.append(/Users/youbinkim/Dropbox (Brown)/yungu/rotd_py)
#os.chdir('../../../')
sys.path.insert(0, '/Users/youbinkim/Dropbox (Brown)/yungu/')

In [2]:
import numpy as np
import ase
#from rotd_py.analysis import *
from rotd_py.analysis import get_micro, get_thermal, get_ej_flux, integrate_micro
from scipy.integrate import simps
import pandas as pd
import rotd_py.rotd_math as rotd_math
import fileinput
#import string

def generate_grid(start, interval, factor, num_point):                          
    #return the grid needed for simulation                                      
    #interval += interval * factor                                              
    #start += interval                                                          
    #return an numpy array with length of num_point                             
    i = 1                                                                       
    grid = [start]                                                              
    for i in range(1, num_point):                                               
        start += interval                                                       
        grid.append(start)                                                      
        interval = interval*factor                                              
    return np.array(grid)                                                       

# temperature, energy grid and angular momentum grid                            
temperature = generate_grid(10, 10, 1.05, 51)                                   
energy = generate_grid(0, 10, 1.05, 169)                                        
angular_mom = generate_grid(0, 1, 1.1, 40) 
#os.getcwd()

In [3]:
e_flux = {}
num_surfaces = 16
for i in range(0, num_surfaces):
    print (i)
    flux = get_micro('surface_data_v0/surface_'+str(i)+'.dat', len(energy), 0)
    e_flux[str(i)]  = flux.copy()

0


FileNotFoundError: [Errno 2] No such file or directory: 'surface_data_v0/surface_0.dat'

In [ ]:
print (e_flux[str(i)])

In [ ]:
data = pd.DataFrame(e_flux)
min_e_flux =  data.min(axis = 1)
print (min_e_flux)

In [ ]:
# This calls out the integrate_micro function implemented in the analysis.py

#print (integrate_micro(min_e_flux, energy, temperature, 9))
test = integrate_micro(min_e_flux, energy, temperature, 9)
print (test)

In [ ]:
tst_out = open('C_surface_data/tst.out').readlines() # This is the C++ data, with output of tst.out
tst_out_me = open('tst_me.out').readlines()

# tst_me.out is the test data made by Xi
# Made to validate the data, which is not necessary any more.
# However, will keep this here, to explain the original overall process

In [ ]:
# This is also needed for data validation
# Only look at the orig_rotd(Since all the validation has been completed,
#                            and we will be using the data created by the analysis.py)

orig_rotd = np.zeros(len(temperature))
me_rotd = np.zeros(len(temperature))

for i in range(0, len(temperature)):
    orig_rotd[i] = float(tst_out[i+273].split()[2])
for i in range(0, len(temperature)):
    me_rotd[i] = float(tst_out_me[i+61].split()[1])

In [ ]:
print (orig_rotd)
print (len(orig_rotd))

In [ ]:
print (me_rotd)
print (len(me_rotd))

In [ ]:
# This is to check whether the array made by the analysis.py has the same 
# number of data and length. Otherwise plot is not possible.

#print (test)
test2 = np.round(test,2)
print (test2)
print (len(test2))
print (len(test))

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
#fig.set_size_inches(4,3.5)
ax = plt.subplot()
ax.plot(temperature, orig_rotd, color ='r', linewidth = 2, label ='C++ rotd data')
ax.plot(temperature, me_rotd, color ='orange', linewidth =2, label = 'Xi rotd data')
ax.plot(temperature, test2, color ='b', linewidth =2, label ='My rotd data')
ax.set_ylabel("Rate constant (cm$^3$s$^{-1}$)",fontsize =12)
ax.set_xlabel("Temperature (K)", fontsize =12)
ax.legend(loc='best')
#fig.savefig('first_code_compare.pdf', bbox_inches='tight')

In [ ]:
# Divide the flux data with avogadro, to obtain the flux.

index =np.where(temperature>300.0)
temperature = temperature[index]
avoga = 6.0221e12
me_rotd = me_rotd[index]/avoga
orig_rotd = orig_rotd[index]/avoga
test3 = test2[index]/avoga

print (me_rotd)
print (orig_rotd)
print (test3)

In [ ]:
fig = plt.figure()
fig.set_size_inches(4,3.5)
ax = plt.subplot(111)

ax.plot(temperature, me_rotd, color='g', linestyle = '-', linewidth=2, label="Xi rotd")
ax.plot(temperature, orig_rotd, color='r', linestyle = '-', linewidth=2, label='C++ rotd')
ax.plot(temperature, test3, color = 'b', linestyle= '-', linewidth=2, label = 'My result')
plus_1 = orig_rotd * 1.05
minus_1 = orig_rotd * 0.95
ax.fill_between(temperature, plus_1, minus_1, facecolor='r', alpha=0.2, label='$\pm$ 5%')
ax.set_ylabel("k (cm$^3$ molecule$^{-1}$s$^{-1}$)",fontsize =12)
#ax.set_yscale('log')
ax.set_xlabel("Temperature (K)", fontsize = 10)   
lgd = ax.legend(loc='best', fontsize=8)
#fig.show()
#fig.savefig('code_compare.pdf', bbox_inches='tight')